In [2]:
from  sklearn.linear_model  import SGDClassifier,LogisticRegression
import evaluation

In [1]:
import preprocess
import featureprocess
import fileio

dfoff,dftest,dfon = fileio.read_all()
dfoff,dfall,train,valid = preprocess.preprocess(dfoff)

dfoff = featureprocess.process_features_main(dfoff)
dfall = featureprocess.process_features_main(dfall)

train = featureprocess.process_features_main(train)
valid = featureprocess.process_features_main(valid)
dftest = featureprocess.process_features_main(valid)



def get_lastfeatures():
    weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
    lastfeatures = ['discount_rate','discount_type','discount_man', 'discount_jian','Distance', 'weekday', 'weekday_type'] + weekdaycols
    lastfeatures = lastfeatures +['只领券', '核销数', '直接买', '单商铺最大购买次数', '成交商家数']
    return lastfeatures

x_train,y_train,x_test,y_test = preprocess.split_xy(train,valid,columns=get_lastfeatures())

In [3]:
model = SGDClassifier(loss='log',penalty='elasticnet',fit_intercept=True,max_iter=100,shuffle=True,
                      alpha = 0.01,l1_ratio = 0.01,n_jobs=1,class_weight=None)

model.fit(x_train, y_train)
y_pre = model.predict(x_test)
y_prob = model.predict_proba(x_test)
#get_model_scores(model,x_test,y_test,y_pre)
evaluation.get_official_auc(y_prob[:,1],y_test,valid)

c:\cx\python\python36\installthere\lib\site-packages\sklearn\metrics\_ranking.py:805: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
c:\cx\python\python36\installthere\lib\site-packages\sklearn\metrics\_ranking.py:89: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):


0.5758439654053648